In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras import Input
import tensorflow_hub as hub
from nltk.corpus import stopwords

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedding_model = hub.load(module_url)
def embed(input):
    return embedding_model(input)

In [3]:
original_data = pd.read_csv("data/mbti_1.csv")
stop_words = stopwords.words('english')
number_of_posts = 5

def remove_links_and_embed(post):
    post = post.lower()
    post = post.split()
    post = [p.strip() for p in post]
    post = [w for w in post if w not in stop_words]
    sample = post[:number_of_posts]
    return embed(sample).numpy()
#     return embed([re.sub("(?P<url>https?://[^\s]+)", '', post).lower().strip() for post in posts.split('|||')])

def make_types_E_vs_I(types):
    return (0 if 'E' in types else 1)

original_data['posts'] = original_data['posts'].apply(remove_links_and_embed)
original_data['type'] = original_data['type'].apply(make_types_E_vs_I)

types = pd.unique(original_data['type'])
code = {tp:i for i,tp in enumerate(types)}
rev_code  = {i:tp for i,tp in enumerate(types)}

for i,person in enumerate(original_data['posts'].values):
    if person.shape != (5,512):
        original_data = original_data.drop([i])

In [95]:
texts_train, texts_test, labels_train, labels_test = train_test_split(np.stack(original_data['posts'].values).reshape(8674,512,5), np.stack(original_data['type'].values), train_size=0.7, random_state=123)

In [113]:
print(np.stack(original_data['posts'].values).reshape(8674,512,5)[0])
np.reshape(np.stack(original_data['posts'].values), (8674*5,512))[0][:5]

[[-0.03862193 -0.05570186 -0.00947765  0.02606708 -0.00523267]
 [ 0.06542242 -0.01833252 -0.06722698  0.00601519  0.08119543]
 [ 0.03176207  0.01511295 -0.00051053 -0.00011399  0.00519196]
 ...
 [-0.05420309 -0.00373369 -0.05336116  0.04266287 -0.02569582]
 [ 0.03040633 -0.08684034  0.04044287 -0.01790301  0.02049469]
 [-0.03231712 -0.01611626 -0.07046355 -0.04581322 -0.00447022]]


array([-0.03862193, -0.05570186, -0.00947765,  0.02606708, -0.00523267],
      dtype=float32)

In [96]:
embedding_dim = 512

inputs = tf.keras.Input(shape=(embedding_dim,5))
a
gru = tf.keras.layers.Bidirectional(
    tf.keras.layers.GRU(
        units=64,
        return_sequences=True
    )
)(inputs)

flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


history = model.fit(
    texts_train,
    labels_train,
    validation_split=0.2,
    batch_size=32,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('./model.h5', save_best_only=True, save_weights_only=True)
    ]
)


Epoch 1/5
152/152 [==============================] - 227s 1s/step - loss: 0.5573 - accuracy: 0.7536 - auc: 0.5343 - val_loss: 0.5391 - val_accuracy: 0.7695 - val_auc: 0.5550
Epoch 2/5
152/152 [==============================] - 233s 1s/step - loss: 0.5240 - accuracy: 0.7728 - auc: 0.6036 - val_loss: 0.5445 - val_accuracy: 0.7695 - val_auc: 0.5332
Epoch 3/5
152/152 [==============================] - 226s 1s/step - loss: 0.5067 - accuracy: 0.7736 - auc: 0.6692 - val_loss: 0.5536 - val_accuracy: 0.7663 - val_auc: 0.5279
Epoch 4/5
152/152 [==============================] - 217s 1s/step - loss: 0.4951 - accuracy: 0.7761 - auc: 0.7002 - val_loss: 0.5608 - val_accuracy: 0.7572 - val_auc: 0.5258
Epoch 5/5
152/152 [==============================] - 223s 1s/step - loss: 0.5001 - accuracy: 0.7690 - auc: 0.7088 - val_loss: 0.5841 - val_accuracy: 0.7613 - val_auc: 0.5325
